In [32]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

## Creating Chill and GDD Features

Chill hours is defined as total hours per day where temperature < 7.2°C using daily min/max temperatures and a triangular approximation.

Growing Degree Days (GDD) are cumulative total of average temperatures being over the threshold, which is temperatures > 10°C.

In [24]:
full_temps_dc = pd.read_csv('Data/full_temps_dc.csv')
full_temps_dc['tavg'] = (full_temps_dc['tmin'] + full_temps_dc['tmax']) / 2
full_temps_dc['date'] = pd.to_datetime(full_temps_dc['date'])

cherry_dc = pd.read_csv("Q_blooms_dc.csv")

In [25]:
def chill_and_gdd(temp_df, bloom_df, chill_thresh = 7.2, gdd_thresh=10):
    results = []

    for year in bloom_df['year'].unique():
        bloom_doy = bloom_df.loc[bloom_df['year'] == year, 'bloom_doy'].values[0]

        # Chill Period: Oct 1 (prev year) -> Feb 28 (current year)
        chill_start = pd.Timestamp(year=year-1, month=10, day=1)
        chill_end = pd.Timestamp(year=year, month=2, day=28)

        chill_data = temp_df[
            (temp_df['date'] >= chill_start) &
            (temp_df['date'] <= chill_end)
        ].copy()

        chill_hours_tot = 0

        for _, row in chill_data.iterrows():
            tmin = row['tmin']
            tmax = row['tmax']

            if tmax < chill_thresh:
                chill_hours = 24
            elif tmin >= chill_thresh:
                chill_hours = 0
            else:
                # Partial day below threshold (Triangular approximation)
                chill_hours = 24 * (chill_thresh - tmin) / (tmax-tmin)
            chill_hours_tot += chill_hours

        # GDD until bloom
        gdd_data = temp_df[
            (temp_df['year'] == year) &
            (temp_df['doy'] <= bloom_doy)
        ]
        gdd_tot = sum(np.maximum(gdd_data['tavg']-gdd_thresh, 0))

        results.append({
            'year': year,
            'chill_hours': chill_hours_tot,
            'gdd': gdd_tot,
            'bloom_doy': bloom_doy
        })
    return pd.DataFrame(results)

model_df = chill_and_gdd(full_temps_dc, cherry_dc)

print(model_df.head())
print(model_df.describe())

   year  chill_hours    gdd  bloom_doy
0  1942  1239.408517  41.05         95
1  1943  2186.390113  70.20         94
2  1944  2111.196179  57.30        100
3  1945  2312.851436  62.85         79
4  1946  2022.637663  69.05         82
              year  chill_hours         gdd   bloom_doy
count    83.000000    83.000000   83.000000   83.000000
mean   1983.000000  1921.259383   60.843976   92.638554
std      24.103942   252.012593   19.905269    7.242453
min    1942.000000  1239.408517   12.700000   74.000000
25%    1962.500000  1773.585214   44.875000   88.000000
50%    1983.000000  1961.704061   59.950000   93.000000
75%    2003.500000  2117.483690   74.975000   97.500000
max    2024.000000  2448.101020  103.450000  108.000000


In [26]:
model_df = model_df.merge(cherry_dc[['year','bloom_doy']], on='year')
print(model_df.corr())
model_df.to_csv('model_df.csv')

                 year  chill_hours       gdd  bloom_doy_x  bloom_doy_y
year         1.000000    -0.378548  0.104073    -0.371851    -0.371851
chill_hours -0.378548     1.000000 -0.349791     0.323713     0.323713
gdd          0.104073    -0.349791  1.000000    -0.366077    -0.366077
bloom_doy_x -0.371851     0.323713 -0.366077     1.000000     1.000000
bloom_doy_y -0.371851     0.323713 -0.366077     1.000000     1.000000


Biologcial machanism is working because gdd is negatively correlated with bloom_doy, and chill_hours has weak, positive correlation with bloom_doy. 

## Basic Model for Feature Investigation

In [27]:
X = model_df[['year','chill_hours', 'gdd']]
y = model_df['bloom_doy']

model = LinearRegression()
model.fit(X, y)

print("Intercept:", model.intercept_)
print("Coefficients:", model.coef_)
print("R^2:", model.score(X,y))

KeyError: 'bloom_doy'

Blood advances ~ 0.09 days per year, thus 0.9 days per decade, approximately 7-8 days ealier since 1942.
This shows an independent long-term advancement trend.

+100 chill hours -> bloom ~0.003 days later.

+10 GDD -> bloom ~ 1.08 days earlier

Heat is driving bloom timing much more than chilling. Spring heat accumulation matters most.


### With Interaction

In [ ]:
model_df['interaction'] = model_df['chill_hours'] * model_df['gdd']

X = model_df[['year', 'chill_hours', 'gdd', 'interaction']]

model.fit(X,y)

print("Intercept:", model.intercept_)
print("Coefficients:", model.coef_)
print("R^2:", model.score(X,y))

Intercept: 273.2204569267005
Coefficients: [-8.78776463e-02  1.64213222e-04 -1.95019588e-01  4.54114381e-05]
R^2: 0.2560535350019685


Chill X Heat interaction is not adding much explanatory power. Barely any improvement in R^2 from the previous model. 

### Polynomial Terms

In [ ]:
model_df['gdd_sq'] = model_df['gdd'] ** 2
model_df['chill_sq'] = model_df['chill_hours'] ** 2

X = model_df[['year', 'chill_hours', 'gdd', 
              'gdd_sq', 'chill_sq'
              ]]

model.fit(X,y)

print("Intercept:", model.intercept_)
print("Coefficients:", model.coef_)
print("R^2:", model.score(X,y))

Intercept: 221.7162898212925
Coefficients: [-9.32323864e-02  7.45336978e-02 -4.75083252e-01  3.01644756e-03
 -1.88701342e-05]
R^2: 0.32747520472360925


Bloom response to heat is nonlinear; likely accelerating with higher GDD. This matches biological forcing models. 

Models for far (based on R^2):
* Biological only: 0.178
* +Year: 0.255
* +Interaction: 0.256
* +Polynomial: 0.327

## Random Forest with Cross-Val

In [ ]:
model_df = pd.read_csv("model_df.csv")
X = model_df[['year', 'chill_hours', 'gdd']]
y = model_df['bloom_doy']

rf = RandomForestRegressor(
    n_estimators=500,
    max_depth=4,
    random_state=42
)

scores = cross_val_score(rf, X, y, cv=10, scoring='r2')

print("CV R^2 scores:", scores)
print("Mean CV R^2:", np.mean(scores))

CV R^2 scores: [-0.30952823  0.06626288  0.20708805  0.25519639 -0.59613754 -0.52067277
  0.19398089 -0.88046908  0.00801942 -0.97397243]
Mean CV R^2: -0.255023241815252


A couple negative mean CV R2 scores meaning RandomForest model is worse than predicting the mean. Linear or polynomial regression is probably better.

## Modeling Change: GDD Threshold

Initial modeling with GDD was using cumulative GDD to predict bloom date, but that caused circular modeling, especially when trying to predict future bloom dates. Now GDD is models as a threshold:
$GDD_{crit} = a + b * Chill$ 

In [ ]:
model_df= chill_and_gdd(full_temps_dc, cherry_dc)

X = model_df[['chill_hours']]
y = model_df['gdd']

gdd_model = LinearRegression()
gdd_model.fit(X,y)

print("Intercept:", gdd_model.intercept_)
print("Slope (Chill effect):", gdd_model.coef_[0])
print("R^2:", gdd_model.score(X,y))

Intercept: 113.92517988566925
Slope (Chill effect): -0.027628338193639957
R^2: 0.12235393062224553


Negative slope means more chill reduces heat requirment, which is biologically correct. Chill accumulation explaning only around 12% of variation in heat requirment. 

## Polynomial Linear Model for 2026 Predictions

### Updating DC 2026 Temperatures Data set

In [ ]:
temps_2026_dc = pd.read_csv("Data/dc_Temps_2026.csv")
temps_2026_dc['date'] = pd.to_datetime(temps_2026_dc['date'])
temps_2026_dc['tavg'] = (temps_2026_dc['tmin'] + temps_2026_dc['tmax']) / 2
temps_2026_dc['year'] = 2026
temps_2026_dc['doy'] = temps_2026_dc['date'].dt.dayofyear

In [ ]:
chill_thresh = 7.2

chill_start = pd.Timestamp(year=2025, month=10, day=1)
chill_end   = pd.Timestamp(year=2026, month=2, day=28)

chill_data = temps_2026_dc[
    (temps_2026_dc['date'] >= chill_start) &
    (temps_2026_dc['date'] <= chill_end)
]

chill_hours_tot = 0

for _, row in chill_data.iterrows():
    tmin = row['tmin']
    tmax = row['tmax']

    if tmax < chill_thresh:
        chill_hours = 24
    elif tmin >= chill_thresh:
        chill_hours = 0
    else:
        chill_hours = 24 * (chill_thresh - tmin) / (tmax - tmin)

    chill_hours_tot += chill_hours

print("2026 Chill Hours:", chill_hours_tot)

2026 Chill Hours: 1183.2473653639247


In [ ]:
required_gdd_2026 = gdd_model.predict([[chill_hours_tot]])
print("Required GDD for 2026:", required_gdd_2026)

Required GDD for 2026: [81.23402151]


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


### 2026 Prediction

In [ ]:
temps_2026_dc = temps_2026_dc.sort_values("date")

gdd_thresh = 10
cumulative_gdd = 0
predicted_date = None

for _, row in temps_2026_dc[temps_2026_dc['year'] == 2026].iterrows():
    
    gdd_day = max(row['tavg'] - gdd_thresh, 0)
    cumulative_gdd += gdd_day
    
    if cumulative_gdd >= required_gdd_2026:
        predicted_date = row['date']
        break

print("Predicted 2026 Bloom Date:", predicted_date)
print("Predicted DOY:", predicted_date.dayofyear)


Predicted 2026 Bloom Date: 2026-04-27 00:00:00
Predicted DOY: 117


This prediction is quite late, but not completely absurd. Before judging it harshly, it will be benefiical to compare the historical distribution of peak bloom dates. 

### Comparison of 2026 Prediction to Historical Distribution

In [ ]:
cherry_dc = pd.read_csv('Q_blooms_dc.csv')

historical_bloom = model_df['bloom_doy']

plt.hist(historical_bloom, bins=15, color='skyblue', edgecolor='black')
plt.axvline(105, color='red', linestyle='--', label='Predicted 2026')
plt.title("Historical Peak Bloom DOY Distribution (Washington, DC)")
plt.xlabel("Day of Year (DOY)")
plt.ylabel("Frequency")
plt.legend()
plt.show()

KeyError: 'bloom_doy'

In [ ]:
percentile = np.mean(historical_bloom <= 105) * 100
print("2026 prediction percentile:", percentile)

2026 prediction percentile: 97.59036144578313


Our 2026 prediction is in the 97th percentile which is extremely high. 

## Modeling Change: GDD Threshold with Extreme Events and Seaonal Variation Accountability 

The simple threshold model captures average relationship between chill hours and bloom timing, but ignores late frosts and spring warming variability. This new hybrid model takes into account extreme weather events in order to correct for late frosts, and accoutns for seasonal variation in spring warming. 

### Frost Risk & Spring Warming Feature Engineering

In [28]:
# Frost risk after dormancy (Feb 15 onward)
post_dormancy = full_temps_dc[full_temps_dc['date'] >= pd.to_datetime('1900-02-15')]  # year ignored for slicing
post_dormancy['frost_flag_27'] = post_dormancy['tmin'] <= 27
post_dormancy['frost_flag_24'] = post_dormancy['tmin'] <= 24

frost_days = post_dormancy.groupby('year')[['frost_flag_27','frost_flag_24']].sum().reset_index()
frost_days.rename(columns={'frost_flag_27':'frost_days_27F','frost_flag_24':'frost_days_24F'}, inplace=True)

model_df = model_df.merge(frost_days, on='year')

# Spring warming rate (linear slope of daily tavg in March)
spring_temp = full_temps_dc[(full_temps_dc['date'].dt.month==3)]
spring_slope = spring_temp.groupby('year').apply(lambda df: np.polyfit(df.index, df['tavg'],1)[0]).reset_index()
spring_slope.columns=['year','spring_slope']
model_df = model_df.merge(spring_slope,on='year')

### Hypbrid Threshold Model Fit

In [29]:
# Threshold model: GDD ~ Chill
X_thresh = model_df[['chill_hours']]
y_thresh = model_df['gdd']
threshold_model = LinearRegression().fit(X_thresh,y_thresh)

# Residuals
model_df['gdd_pred'] = threshold_model.predict(X_thresh)
model_df['residuals'] = model_df['bloom_doy'] - model_df['gdd_pred']

# Hybrid correction
X_resid = model_df[['frost_days_27F','frost_days_24F','spring_slope']]
resid_model = LinearRegression().fit(X_resid, model_df['residuals'])
model_df['resid_pred'] = resid_model.predict(X_resid)

# Final historical predicted bloom DOY
model_df['bloom_pred'] = model_df['gdd_pred'] + model_df['resid_pred']

KeyError: 'bloom_doy'

### New 2026 Predictions

In [30]:
# Chill hours
chill_start = pd.Timestamp(year=2025, month=10, day=1)
chill_end   = pd.Timestamp(year=2026, month=2, day=28)
chill_data = temps_2026_dc[(temps_2026_dc['date']>=chill_start) & (temps_2026_dc['date']<=chill_end)]
chill_hours_2026 = sum([
    24 if row['tmax']<chill_thresh else 0 if row['tmin']>=chill_thresh else 24*(chill_thresh-row['tmin'])/(row['tmax']-row['tmin'])
    for _, row in chill_data.iterrows()
])

# Frost risk
post_dorm_2026 = temps_2026_dc[temps_2026_dc['date'] >= pd.Timestamp(year=2026, month=2, day=15)]
frost_days_27_2026 = (post_dorm_2026['tmin'] <= 27).sum()
frost_days_24_2026 = (post_dorm_2026['tmin'] <= 24).sum()

# Spring warming rate (linear slope March tavg)
spring_2026 = temps_2026_dc[temps_2026_dc['date'].dt.month==3]
spring_slope_2026 = np.polyfit(spring_2026.index, spring_2026['tavg'],1)[0]

# Threshold model
required_gdd_2026 = threshold_model.predict([[chill_hours_2026]])[0]

# Hybrid residual correction
resid_corr_2026 = resid_model.predict([[frost_days_27_2026, frost_days_24_2026, spring_slope_2026]])[0]

# Predicted bloom DOY
predicted_bloom_DOY_2026 = required_gdd_2026 + resid_corr_2026
print("Predicted 2026 bloom DOY:", predicted_bloom_DOY_2026)

# Optionally, find the calendar date
predicted_date_2026 = pd.Timestamp(year=2026, month=1, day=1) + pd.Timedelta(days=int(predicted_bloom_DOY_2026)-1)
print("Predicted 2026 bloom date:", predicted_date_2026)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


NameError: name 'resid_model' is not defined